In [54]:
import pandas as pd
import numpy as np
from collections import OrderedDict

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_info_columns',1000)

In [55]:
application_test = pd.read_csv('../application_test.csv')
application_train  = pd.read_csv('../application_train.csv')
bureau = pd.read_csv('../bureau.csv')
bureau_balance = pd.read_csv('../bureau_balance.csv')
credit_card_balance = pd.read_csv('../credit_card_balance.csv')
installments_payments = pd.read_csv('../installments_payments.csv')
POS_CASH_balance = pd.read_csv('../POS_CASH_balance.csv')
previous_application = pd.read_csv('../previous_application.csv')

In [56]:
print ('the count of {} is {},approximately {} thousands'.format('application_test',application_test.shape[0],application_test.shape[0]/1000))
print ('the count of {} is {},approximately {} thousands'.format('application_train',application_train.shape[0],application_train.shape[0]/1000))
print ('the count of {} is {},approximately {} thousands'.format('bureau',bureau.shape[0],bureau.shape[0]/1000))
print ('the count of {} is {},approximately {} thousands'.format('bureau_balance',bureau_balance.shape[0],bureau_balance.shape[0]/1000))
print ('the count of {} is {},approximately {} thousands'.format('credit_card_balance',credit_card_balance.shape[0],credit_card_balance.shape[0]/1000))
print ('the count of {} is {},approximately {} thousands'.format('installments_payments',installments_payments.shape[0],installments_payments.shape[0]/1000))
print ('the count of {} is {},approximately {} thousands'.format('POS_CASH_balance',POS_CASH_balance.shape[0],POS_CASH_balance.shape[0]/1000))
print ('the count of {} is {},approximately {} thousands'.format('previous_application',previous_application.shape[0],previous_application.shape[0]/1000))

the count of application_test is 48744,approximately 48 thousands
the count of application_train is 307511,approximately 307 thousands
the count of bureau is 1716428,approximately 1716 thousands
the count of bureau_balance is 27299925,approximately 27299 thousands
the count of credit_card_balance is 3840312,approximately 3840 thousands
the count of installments_payments is 13605401,approximately 13605 thousands
the count of POS_CASH_balance is 10001358,approximately 10001 thousands
the count of previous_application is 1670214,approximately 1670 thousands


### remain the meta data for train and test

In [57]:
train_target_sequence = application_train[['SK_ID_CURR','TARGET']]
test_sequence = application_test['SK_ID_CURR']
application_train = application_train.drop(['TARGET'], axis=1)

In [58]:
application_train.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

### put the test and train together

In [59]:
application_all = application_train.append(application_test)
print ('all dataset counts is {}'.format(application_all.shape[0]))

all dataset counts is 356255


### do some regular processes for the data

1. drop the the columns with more than 30 percent NA values
2. drop the unnecessary columns
3. fillNa by when int/float then mean, when object then mode 

In [60]:
def miss_ratio_dict(df):
    """
    calculate the miss data ratio
    :param df:DataFrame
    :return: a dict which the columns and its miss precentage
    """
    columns_NA_ratio = OrderedDict()
    for columns in df.columns:
        df_na = df[columns].isnull()
        na_ratio =float(df_na[df_na==True].shape[0])/float(df_na.shape[0])
        columns_NA_ratio[columns] = round(na_ratio*100,2)
    return columns_NA_ratio

In [61]:
def fillna_values_dict(df):
    """
    get the fillna values for each columns
    if dtype is numeric,then fillna values is mean
    if dtype is object,then fillna value is mode
    :param df:
    :return:dict,k is columns,v is fillna value
    """
    columns_NA_fill = OrderedDict()
    for columns in df.columns:
        dtypes = df[columns].dtype.name
        if 'int' in dtypes or 'float' in dtypes:
            columns_NA_fill[columns] = df[columns].mean()
            continue
        if dtypes == 'object':
            columns_NA_fill[columns] = df[columns].mode()[0]
            continue
        print ('{} is not found dtypes'.format(dtypes))
    return columns_NA_fill

In [62]:
datasets = [bureau,bureau_balance,credit_card_balance,installments_payments,POS_CASH_balance,previous_application]

In [63]:
for ds in datasets:
    miss_ratio = miss_ratio_dict(ds)
    #print (miss_ratio)
    drop_columns = list()
    for column in ds.columns:
        if miss_ratio[column] > 20:
            drop_columns.append(column)
    print (drop_columns)
    ds.drop(drop_columns, axis=1, inplace=True)     

['DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'AMT_CREDIT_SUM_LIMIT', 'AMT_ANNUITY']
[]
[]
[]
[]
['AMT_ANNUITY', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'NAME_TYPE_SUITE', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']


In [64]:
for ds in datasets:
    fillna_values = fillna_values_dict(ds)
    print (fillna_values)
    ds.fillna(fillna_values, inplace=True)

OrderedDict([('SK_ID_CURR', 278214.9336447553), ('SK_ID_BUREAU', 5924434.489031873), ('CREDIT_ACTIVE', 'Closed'), ('CREDIT_CURRENCY', 'currency 1'), ('DAYS_CREDIT', -1142.107684680045), ('CREDIT_DAY_OVERDUE', 0.8181665645165425), ('DAYS_CREDIT_ENDDATE', 510.51736230309615), ('CNT_CREDIT_PROLONG', 0.00641040579622332), ('AMT_CREDIT_SUM', 354994.59191768744), ('AMT_CREDIT_SUM_DEBT', 137085.11995216075), ('AMT_CREDIT_SUM_OVERDUE', 37.91275774165883), ('CREDIT_TYPE', 'Consumer credit'), ('DAYS_CREDIT_UPDATE', -593.7483197663986)])
OrderedDict([('SK_ID_BUREAU', 6036297.332973552), ('MONTHS_BALANCE', -30.74168665298531), ('STATUS', 'C')])
OrderedDict([('SK_ID_PREV', 1904503.589899727), ('SK_ID_CURR', 278324.20728888694), ('MONTHS_BALANCE', -34.52192061478338), ('AMT_BALANCE', 58300.15526185774), ('AMT_CREDIT_LIMIT_ACTUAL', 153807.95740033622), ('AMT_DRAWINGS_ATM_CURRENT', 5961.324822059312), ('AMT_DRAWINGS_CURRENT', 7433.388179202412), ('AMT_DRAWINGS_OTHER_CURRENT', 288.16958242463346), ('AM

### let's create some feature joining with main table

In [65]:
bureau.head(5)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE
0,215354,5714462,Closed,currency 1,-497,0,-153.000000,0,91323.0,0.000000,0.0,Consumer credit,-131
1,215354,5714463,Active,currency 1,-208,0,1075.000000,0,225000.0,171342.000000,0.0,Credit card,-20
2,215354,5714464,Active,currency 1,-203,0,528.000000,0,464323.5,137085.119952,0.0,Consumer credit,-16
3,215354,5714465,Active,currency 1,-203,0,510.517362,0,90000.0,137085.119952,0.0,Credit card,-16
4,215354,5714466,Active,currency 1,-629,0,1197.000000,0,2700000.0,137085.119952,0.0,Consumer credit,-21


### feature from bureau

In [66]:
# the counts of Activce,Closed,Bad debt,Sold credit from bureau where application occurred
bureau_status = pd.get_dummies(bureau['CREDIT_ACTIVE'], prefix='bureau_status')
bureau_status = pd.concat([bureau['SK_ID_CURR'], bureau_status], axis=1)
bureau_status = bureau_status.groupby(['SK_ID_CURR'], as_index =False).sum()
bureau_status.columns = ['SK_ID_CURR','bureau_status_Active_counts','bureau_status_Bad debt_counts','bureau_status_Closed_counts','bureau_status_Sold_counts']
bureau_status.head(5)

,SK_ID_CURR,bureau_status_Active_counts,bureau_status_Bad debt_counts,bureau_status_Closed_counts,bureau_status_Sold_counts
0,100001,3,0,4,0
1,100002,2,0,6,0
2,100003,1,0,3,0
3,100004,0,0,2,0
4,100005,2,0,1,0


In [67]:
# the counts of credits from bureau
bureau_counts = bureau[['SK_ID_CURR','SK_ID_BUREAU']].groupby(['SK_ID_CURR'], as_index =False).count()
bureau_counts.columns =  ['SK_ID_CURR','BUREAU_COUNTS']
bureau_counts.head(5)

,SK_ID_CURR,BUREAU_COUNTS
0,100001,7
1,100002,8
2,100003,4
3,100004,2
4,100005,3


In [68]:
# the totoal sum feature
bureau_sum = bureau[['SK_ID_CURR','DAYS_CREDIT','CREDIT_DAY_OVERDUE','CNT_CREDIT_PROLONG','AMT_CREDIT_SUM','AMT_CREDIT_SUM_OVERDUE','DAYS_CREDIT_UPDATE']]
bureau_sum = bureau_sum.groupby(['SK_ID_CURR'], as_index=False).sum()

def addPrefix_bureau(x):
    if x!='SK_ID_CURR':
        return 'BUREAU_'+x
    return x
    
bureau_sum.columns = bureau_sum.columns.map(addPrefix_bureau)
bureau_sum.head(3)

,SK_ID_CURR,BUREAU_DAYS_CREDIT,BUREAU_CREDIT_DAY_OVERDUE,BUREAU_CNT_CREDIT_PROLONG,BUREAU_AMT_CREDIT_SUM,BUREAU_AMT_CREDIT_SUM_OVERDUE,BUREAU_DAYS_CREDIT_UPDATE
0,100001,-5145,0,0,1453365.000,0.0,-652
1,100002,-6992,0,0,865055.565,0.0,-3999
2,100003,-5603,0,0,1017400.500,0.0,-3264


### feature from POS_CASH_balance

In [69]:
POS_CASH_balance.sort_values(by='SK_ID_CURR').head(10)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
8531326,1851984,100001,-94,4.0,0.0,Active,0,0
1891462,1851984,100001,-95,4.0,1.0,Active,7,7
8789081,1369693,100001,-56,4.0,3.0,Active,0,0
4704415,1369693,100001,-54,4.0,1.0,Active,0,0
7167007,1369693,100001,-57,4.0,4.0,Active,0,0
4928574,1851984,100001,-93,4.0,0.0,Completed,0,0
1261679,1851984,100001,-96,4.0,2.0,Active,0,0
7823681,1369693,100001,-55,4.0,2.0,Active,0,0
2197888,1369693,100001,-53,4.0,0.0,Completed,0,0
9849444,1038818,100002,-15,24.0,20.0,Active,0,0


In [70]:
#count for each of our sample
pos_cash_balance_count = POS_CASH_balance[['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR'], as_index=False).count()
pos_cash_balance_count.columns =['SK_ID_CURR','POS_CASH_balance_counts']
pos_cash_balance_count.head(5)

,SK_ID_CURR,POS_CASH_balance_counts
0,100001,9
1,100002,19
2,100003,28
3,100004,4
4,100005,11


In [71]:
#sum feature
pos_cash_balance_sum = POS_CASH_balance[['SK_ID_CURR','SK_DPD','SK_DPD_DEF']].groupby(['SK_ID_CURR'], as_index=False).sum()
pos_cash_balance_sum.columns = ['SK_ID_CURR','POS_SK_DPD_SUM','POS_SK_DPD_DEF_SUM']
pos_cash_balance_sum.head(5)

,SK_ID_CURR,POS_SK_DPD_SUM,POS_SK_DPD_DEF_SUM
0,100001,7,7
1,100002,0,0
2,100003,0,0
3,100004,0,0
4,100005,0,0


### feature from credit_card_balance

In [72]:
credit_card_balance.head(5)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [73]:
#count and sum
credit_card_balance_count = credit_card_balance[['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR'], as_index=False).count()
credit_card_balance_count.columns = ['SK_ID_CURR','CREDIT_CARD_COUNTS']

credit_card_balance_sum = credit_card_balance[['SK_ID_CURR','SK_DPD','SK_DPD_DEF']].groupby(['SK_ID_CURR'], as_index=False).sum()
credit_card_balance_sum.columns = ['SK_ID_CURR','CREDIT_CARD_SK_DPD_SUM','CREDIT_CARD_SK_DPD_DEF_SUM']

In [74]:
credit_card_balance_count.head(5)

,SK_ID_CURR,CREDIT_CARD_COUNTS
0,100006,6
1,100011,74
2,100013,96
3,100021,17
4,100023,8


In [75]:
credit_card_balance_sum.head(5)

,SK_ID_CURR,CREDIT_CARD_SK_DPD_SUM,CREDIT_CARD_SK_DPD_DEF_SUM
0,100006,0,0
1,100011,0,0
2,100013,1,1
3,100021,0,0
4,100023,0,0


### feature from installments_payments

In [76]:
installments_payments.sort_values(by='SK_ID_CURR').head(20)

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
3458712,1369693,100001,1.0,3,-1649.0,-1660.0,3951.000,3951.000
3435373,1851984,100001,1.0,4,-2856.0,-2856.0,3980.925,3980.925
2624024,1369693,100001,2.0,4,-1619.0,-1628.0,17397.900,17397.900
1761012,1851984,100001,1.0,2,-2916.0,-2916.0,3982.050,3982.050
3774071,1851984,100001,1.0,3,-2886.0,-2875.0,3982.050,3982.050
1478621,1369693,100001,1.0,1,-1709.0,-1715.0,3951.000,3951.000
2568722,1369693,100001,1.0,2,-1679.0,-1715.0,3951.000,3951.000
2163032,1038818,100002,1.0,2,-535.0,-562.0,9251.775,9251.775
504426,1038818,100002,1.0,12,-235.0,-250.0,9251.775,9251.775
699804,1038818,100002,1.0,13,-205.0,-224.0,9251.775,9251.775


In [77]:
# installments counts
installments_payments_counts = installments_payments[['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR'], as_index=False).count()
installments_payments_counts.columns = ['SK_ID_CURR','INSTALLMENTS_COUNTS']
installments_payments_counts.head(5)

,SK_ID_CURR,INSTALLMENTS_COUNTS
0,100001,7
1,100002,19
2,100003,25
3,100004,3
4,100005,9


### feature from previous_application

In [78]:
# we drop the data which previous constract status is between Canceled and unused offer
previous_application = previous_application[previous_application['NAME_CONTRACT_STATUS'] != 'Canceled']
previous_application = previous_application[previous_application['NAME_CONTRACT_STATUS'] != 'Unused offer']

In [79]:
previous_application.head(5)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_APPLICATION,AMT_CREDIT,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,NAME_YIELD_GROUP,PRODUCT_COMBINATION
0,2030495,271877,Consumer loans,17145.0,17145.0,SATURDAY,15,Y,1,XAP,Approved,-73,Cash through the bank,XAP,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,middle,POS mobile with interest
1,2802425,108129,Cash loans,607500.0,679671.0,THURSDAY,11,Y,1,XNA,Approved,-164,XNA,XAP,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,low_action,Cash X-Sell: low
2,2523466,122040,Cash loans,112500.0,136444.5,TUESDAY,11,Y,1,XNA,Approved,-301,Cash through the bank,XAP,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,high,Cash X-Sell: high
3,2819243,176158,Cash loans,450000.0,470790.0,MONDAY,7,Y,1,XNA,Approved,-512,Cash through the bank,XAP,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,middle,Cash X-Sell: middle
4,1784265,202054,Cash loans,337500.0,404055.0,THURSDAY,9,Y,1,Repairs,Refused,-781,Cash through the bank,HC,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,high,Cash Street: high


In [80]:
# contract type counts in previous application
previous_contract_type_counts = pd.get_dummies(previous_application['NAME_CONTRACT_TYPE'], prefix='PREVIOUS')
previous_contract_type_counts.drop(['PREVIOUS_XNA'], axis=1, inplace=True)
previous_contract_type_counts = pd.concat([previous_application['SK_ID_CURR'],previous_contract_type_counts], axis=1)
previous_contract_type_counts = previous_contract_type_counts.groupby(['SK_ID_CURR'], as_index=False).sum()
previous_contract_type_counts.columns = ['SK_ID_CURR','PREVIOUS_Cash loans_counts','PREVIOUS_Consumer loans_counts','PREVIOUS_Revolving loans_counts']
previous_contract_type_counts.head(5)

,SK_ID_CURR,PREVIOUS_Cash loans_counts,PREVIOUS_Consumer loans_counts,PREVIOUS_Revolving loans_counts
0,100001,0,1,0
1,100002,0,1,0
2,100003,1,2,0
3,100004,0,1,0
4,100005,0,1,0


In [81]:
# weekday counts in previous application
previous_application_weekday_start = pd.get_dummies(previous_application['WEEKDAY_APPR_PROCESS_START'], prefix='PREVIOUS')
previous_application_weekday_start = pd.concat([previous_application['SK_ID_CURR'],previous_application_weekday_start], axis=1)
previous_application_weekday_start = previous_application_weekday_start.groupby(['SK_ID_CURR'], as_index=False).sum()
def addSuffix_count(x):
    if x!='SK_ID_CURR':
        return x + '_COUNTS'
    return x
previous_application_weekday_start.columns = previous_application_weekday_start.columns.map(addSuffix_count)

previous_application_weekday_start.head(5)

,SK_ID_CURR,PREVIOUS_FRIDAY_COUNTS,PREVIOUS_MONDAY_COUNTS,PREVIOUS_SATURDAY_COUNTS,PREVIOUS_SUNDAY_COUNTS,PREVIOUS_THURSDAY_COUNTS,PREVIOUS_TUESDAY_COUNTS,PREVIOUS_WEDNESDAY_COUNTS
0,100001,1,0,0,0,0,0,0
1,100002,0,0,1,0,0,0,0
2,100003,1,0,1,1,0,0,0
3,100004,1,0,0,0,0,0,0
4,100005,0,0,0,0,1,0,0


In [82]:
# approved and refused counts in previous applications
previous_status_counts = pd.get_dummies(previous_application['NAME_CONTRACT_STATUS'])
previous_status_counts = pd.concat([previous_application['SK_ID_CURR'],previous_status_counts], axis=1)
previous_status_counts = previous_status_counts.groupby(['SK_ID_CURR'], as_index=False).sum()
previous_status_counts.columns = ['SK_ID_CURR','PREVIOUS_APPROVED_COUNTS','PREVIOUS_REFUSED_COUNTS']
previous_status_counts.head(5)

,SK_ID_CURR,PREVIOUS_APPROVED_COUNTS,PREVIOUS_REFUSED_COUNTS
0,100001,1,0
1,100002,1,0
2,100003,3,0
3,100004,1,0
4,100005,1,0


In [83]:
# previous application counts
previous_application_counts = previous_application[['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR'], as_index=False).count()
previous_application_counts.columns = ['SK_ID_CURR','PREVIOUS_APPLICATION_COUNTS']
previous_application_counts.head(5)

,SK_ID_CURR,PREVIOUS_APPLICATION_COUNTS
0,100001,1
1,100002,1
2,100003,3
3,100004,1
4,100005,1


### merge the new feature above with application sample

In [84]:
feature_data = [bureau_status,bureau_counts,bureau_sum,pos_cash_balance_count,pos_cash_balance_sum,credit_card_balance_count,installments_payments_counts,previous_contract_type_counts,previous_application_weekday_start,previous_status_counts,previous_application_counts]
application_merge = pd.concat([application_all['SK_ID_CURR'],application_all['NAME_CONTRACT_TYPE']], axis=1)
for d in feature_data:
     application_merge = pd.merge(left=application_merge, right=d, how='left', on='SK_ID_CURR')

application_merge.drop(['NAME_CONTRACT_TYPE'], axis=1, inplace=True)
application_merge.fillna(0, inplace=True）

SyntaxError: invalid syntax (<ipython-input-84-382a8e2d409d>, line 7)

In [ ]:
application_merage.to_csv('../application_assemble.csv')